# INDR344 HW05: Emergency Room

# Özge Sayar

In [66]:
import numpy as np
import simpy as sp

In [67]:
def generate_interarrival_time():
    interarrival_time = np.random.randint(21,60)
    return(interarrival_time)

def generate_service_time_NIA_patient_3():
    generate_service_time_NIA_patient_3 = np.random.randint(3,78)
    return(generate_service_time_NIA_patient_3)

def generate_service_time_NIA_patient_2():
    generate_service_time_NIA_patient_2 = np.random.randint(5,56)
    return(generate_service_time_NIA_patient_2)

def generate_service_time_CW_patient_1():
    generate_service_time_CW_patient_1 = np.random.randint(1,30)
    return(generate_service_time_CW_patient_1)

def generate_service_time_CW_patient_2():
    generate_service_time_CW_patient_2 = np.random.randint(2,19)
    return(generate_service_time_CW_patient_2)

#### Simulation with 2 doctors

In [68]:
Total_time_to_first_doctor_NIA_patients = []
Total_time_to_first_doctor_CW_patients = []
Total_time_to_second_doctor_for_NIA_patients = []
Total_time_to_second_doctor_for_CW_patients = []
Total_time_to_discharge_for_NIA_patients = []
Total_time_to_discharge_for_CW_patients = []
Total_queue_length = []
Time_spent_with_doctors = []
Total_doctor_utilization = []

def patient(env, doctor, d):
    global Total_time_to_first_doctor_NIA_patients
    global Total_time_to_first_doctor_CW_patients
    global Total_time_to_second_doctor_for_NIA_patients
    global Total_time_to_second_doctor_for_CW_patients
    global Total_time_to_discharge_for_NIA_patients
    global Total_time_to_discharge_for_CW_patients
    global Total_doctor_utilization
    global Total_queue_length
    
    NIA_patient = False
    CW_patient = False
    
    Total_queue_length.append(len(doctor.queue))
    
    if np.random.rand() < 0.20:
        NIA_patient = True 
    else:
        CW_patient = True 
    
    if NIA_patient:
        patient_request = doctor.request(-2)
    elif CW_patient:
        patient_request = doctor.request(0)

    patient_arrival_time = env.now
    Total_queue_length.append(len(doctor.queue))

    yield patient_request
    patient_enter_time = env.now
    Total_queue_length.append(len(doctor.queue))
    
    if NIA_patient:
        Total_time_to_first_doctor_NIA_patients.append(patient_enter_time - patient_arrival_time) 
        service_time = generate_service_time_NIA_patient_3()
    elif CW_patient:
        Total_time_to_first_doctor_CW_patients.append(patient_enter_time - patient_arrival_time)
        service_time = generate_service_time_CW_patient_1()
    yield env.timeout(service_time)
    first_doctor_exit_time = env.now
    Time_spent_with_doctors.append(service_time) 
    
    doctor.release(patient_request) 
    
    Total_queue_length.append(len(doctor.queue))    
    patient_request_again = doctor.request(-1)
    
    Total_queue_length.append(len(doctor.queue))
    
    yield(patient_request_again)
    patient_enter_time_second = env.now
    Total_queue_length.append(len(doctor.queue))
    
    if NIA_patient:
        Total_time_to_second_doctor_for_NIA_patients.append(patient_enter_time_second - patient_arrival_time) 
        second_service_time = generate_service_time_NIA_patient_2()
    elif CW_patient:
        Total_time_to_second_doctor_for_CW_patients.append(patient_enter_time_second - patient_arrival_time)
        second_service_time = generate_service_time_CW_patient_2()
    
    Total_queue_length.append(len(doctor.queue))
    yield env.timeout(second_service_time)
    Time_spent_with_doctors.append(second_service_time) 

    doctor.release(patient_request_again)
    
    patient_exit_time = env.now
    Total_queue_length.append(len(doctor.queue))
    
    if NIA_patient:
        Total_time_to_discharge_for_NIA_patients.append(patient_exit_time - patient_arrival_time) 
    elif CW_patient:
        Total_time_to_discharge_for_CW_patients.append(patient_exit_time - patient_arrival_time)

    Total_doctor_utilization = np.sum(Time_spent_with_doctors)/(24*60*20*d)
    
def start_collecting(env, time, doctor):
    yield env.timeout(time)
    Total_time_to_first_doctor_NIA_patients.clear() 
    Total_time_to_first_doctor_CW_patients.clear() 
    Total_time_to_second_doctor_for_NIA_patients.clear() 
    Total_time_to_second_doctor_for_CW_patients.clear() 
    Total_time_to_discharge_for_NIA_patients.clear() 
    Total_time_to_discharge_for_CW_patients.clear() 
    Time_spent_with_doctors.clear() 
    Total_queue_length.clear()
    Total_queue_length.append(len(doctor.queue))

def setup(env,d):
    doctor = sp.PriorityResource(env, d)
    env.process(start_collecting(env, 48*60, doctor))
    
    while True:
        interarrival_time = generate_interarrival_time()
        yield env.timeout(interarrival_time)
        env.process(patient(env, doctor, d))
    
np.random.seed(344)
env = sp.Environment()
env.process(setup(env,2))
env.run(until = 24*60*22)
    

print("--------------------------------------------------------------------------------------------------------------")
print("Average time to first doctor for NIA patients", np.around(np.mean(Total_time_to_first_doctor_NIA_patients), decimals=4))
print("Average time to first doctor for CW patients", np.around(np.mean(Total_time_to_first_doctor_CW_patients), decimals=4))
print("Average time to second doctor for NIA patients", np.around(np.mean(Total_time_to_second_doctor_for_NIA_patients), decimals=4))
print("Average time to second doctor for CW patients", np.around(np.mean(Total_time_to_second_doctor_for_CW_patients), decimals=4))
print("Average time to discharge for NIA patients", np.around(np.mean(Total_time_to_discharge_for_NIA_patients), decimals=4))
print("Average time to discharge for CW patients", np.around(np.mean(Total_time_to_discharge_for_CW_patients), decimals=4))
print("--------------------------------------------------------------------------------------------------------------")
print("Average doctor utilization", np.around(Total_doctor_utilization, decimals=4))
print("Average queue length", np.around(np.mean(Total_queue_length), decimals = 4))
print("--------------------------------------------------------------------------------------------------------------")


--------------------------------------------------------------------------------------------------------------
Average time to first doctor for NIA patients 0.7973
Average time to first doctor for CW patients 0.8014
Average time to second doctor for NIA patients 43.2857
Average time to second doctor for CW patients 15.8815
Average time to discharge for NIA patients 74.4218
Average time to discharge for CW patients 25.7805
--------------------------------------------------------------------------------------------------------------
Average doctor utilization 0.4358
Average queue length 0.0317
--------------------------------------------------------------------------------------------------------------


#### Simulation with 1 doctor 

In [69]:
np.random.seed(344)
env = sp.Environment()
env.process(setup(env,1))
env.run(until = 24*60*22)
    

print("--------------------------------------------------------------------------------------------------------------")
print("Average time to first doctor for NIA patients", np.around(np.mean(Total_time_to_first_doctor_NIA_patients), decimals=4))
print("Average time to first doctor for CW patients", np.around(np.mean(Total_time_to_first_doctor_CW_patients), decimals=4))
print("Average time to second doctor for NIA patients", np.around(np.mean(Total_time_to_second_doctor_for_NIA_patients), decimals=4))
print("Average time to second doctor for CW patients", np.around(np.mean(Total_time_to_second_doctor_for_CW_patients), decimals=4))
print("Average time to discharge for NIA patients", np.around(np.mean(Total_time_to_discharge_for_NIA_patients), decimals=4))
print("Average time to discharge for CW patients", np.around(np.mean(Total_time_to_discharge_for_CW_patients), decimals=4))
print("--------------------------------------------------------------------------------------------------------------")
print("Average doctor utilization", np.around(Total_doctor_utilization, decimals=4))
print("Average queue length", np.around(np.mean(Total_queue_length), decimals = 4))
print("--------------------------------------------------------------------------------------------------------------")

--------------------------------------------------------------------------------------------------------------
Average time to first doctor for NIA patients 11.4558
Average time to first doctor for CW patients 101.5439
Average time to second doctor for NIA patients 80.449
Average time to second doctor for CW patients 119.0771
Average time to discharge for NIA patients 110.7211
Average time to discharge for CW patients 128.7356
--------------------------------------------------------------------------------------------------------------
Average doctor utilization 0.8536
Average queue length 2.3582
--------------------------------------------------------------------------------------------------------------


#### Simulation with 3 doctors 

In [70]:
np.random.seed(344)
env = sp.Environment()
env.process(setup(env,3))
env.run(until = 24*60*22)
    

print("--------------------------------------------------------------------------------------------------------------")
print("Average time to first doctor for NIA patients", np.around(np.mean(Total_time_to_first_doctor_NIA_patients), decimals=4))
print("Average time to first doctor for CW patients", np.around(np.mean(Total_time_to_first_doctor_CW_patients), decimals=4))
print("Average time to second doctor for NIA patients", np.around(np.mean(Total_time_to_second_doctor_for_NIA_patients), decimals=4))
print("Average time to second doctor for CW patients", np.around(np.mean(Total_time_to_second_doctor_for_CW_patients), decimals=4))
print("Average time to discharge for NIA patients", np.around(np.mean(Total_time_to_discharge_for_NIA_patients), decimals=4))
print("Average time to discharge for CW patients", np.around(np.mean(Total_time_to_discharge_for_CW_patients), decimals=4))
print("--------------------------------------------------------------------------------------------------------------")
print("Average doctor utilization", np.around(Total_doctor_utilization, decimals=4))
print("Average queue length", np.around(np.mean(Total_queue_length), decimals = 4))
print("--------------------------------------------------------------------------------------------------------------")

--------------------------------------------------------------------------------------------------------------
Average time to first doctor for NIA patients 0.0
Average time to first doctor for CW patients 0.0087
Average time to second doctor for NIA patients 42.651
Average time to second doctor for CW patients 15.007
Average time to discharge for NIA patients 74.8446
Average time to discharge for CW patients 24.92
--------------------------------------------------------------------------------------------------------------
Average doctor utilization 0.2941
Average queue length 0.0003
--------------------------------------------------------------------------------------------------------------
